In [ ]:

import kagglehub

import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import _LRScheduler

from tqdm import tqdm  # For progress bar

tusimple_path = kagglehub.dataset_download('manideep1108/tusimple')
tusimple_path

'/kaggle/input/tusimple'

In [ ]:
# Global Variables
DATASET_BASE_PATH = os.path.join(tusimple_path, "TUSimple")
N_CLASSES_SEGMENTATION = 5 # for a maximum of 5 lanes
# hyper parameters
BATCH_SIZE = 16
N_EPOCHS = 150
LEARNING_RATE = 5e-4
N_SAMPLES = None
WIDTH = 512
HEIGHT = 256

In [ ]:
class LoadDataset(torch.utils.data.Dataset):
    def __init__(self, train=True, size=(360, 640), num_samples=None):
        self._base_path = DATASET_BASE_PATH
        self._image_size = size
        self._mode = "train" if train else "test"
        self._file = "train_val_gt.txt" if train else "test_gt.txt"
        self._data = []
        self.num_samples = num_samples  # Number of samples to load (None for all)
        self._process_image_path()

    def _process_image_path(self):
        folder = "train_set" if self._mode == "train" else "test_set"
        list_file_path = os.path.join(self._base_path, "train_set/seg_label/list/", self._file)

        with open(file=list_file_path) as file:
            for lines in file:
                line = lines.strip().split()
                img_path = os.path.join(self._base_path, folder, line[0][1:])
                lanes_path = os.path.join(self._base_path, "train_set", line[1][1:])
                flags = [int(i) for i in line[2:]]
                self._data.append([img_path, lanes_path, flags])

        # Reduce the dataset if num_samples is specified
        if self.num_samples is not None:
            self._data = self._data[:self.num_samples]

    def __len__(self):
        return len(self._data)

    def __getitem__(self, idx):
        image_path = self._data[idx][0]
        image = cv2.imread(image_path)
        h, w, c = image.shape

        # original image
        original_image = image

        # resized image
        image = cv2.resize(image, self._image_size)  # resize accepts as (width, height)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # lanes image
        lanes_image = cv2.imread(self._data[idx][1])
        lanes_image = lanes_image[:, :, 1]  # lane info is in the second channel
        lanes_image = cv2.resize(lanes_image, self._image_size)  # resize accepts as (width, height)


        # Convert images to PyTorch tensors - optimized for deep learning
        image = torch.from_numpy(image).float().permute((2,0,1))
        lanes_image = torch.from_numpy(lanes_image.copy())
        flags = torch.as_tensor(self._data[idx][2])

        input = {
            "image": image,
            "lanes_image": lanes_image,
            "flags": flags
        }

        return input


In [ ]:
dataset = LoadDataset(size=(WIDTH, HEIGHT),num_samples= N_SAMPLES)
train_size = int(0.80 * len(dataset))
train_ds, val_ds = torch.utils.data.random_split(dataset, [train_size, len(dataset)-train_size])

print(train_ds.__len__())
print(val_ds.__len__())

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=4)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=4)


2900
726


In [ ]:
train_ds[0]['image'].shape

torch.Size([3, 256, 512])

In [ ]:
# ERFNet full model definition for Pytorch
# Sept 2017
# Eduardo Romera
#######################


class DownsamplerBlock (nn.Module):
    def __init__(self, ninput, noutput):
        super().__init__()

        self.conv = nn.Conv2d(ninput, noutput-ninput, (3, 3), stride=2, padding=1, bias=True)
        self.pool = nn.MaxPool2d(2, stride=2)
        self.bn = nn.BatchNorm2d(noutput, eps=1e-3)

    def forward(self, input):
        output = torch.cat([self.conv(input), self.pool(input)], 1)
        output = self.bn(output)
        return F.relu(output)


class non_bottleneck_1d (nn.Module):
    def __init__(self, chann, dropprob, dilated):
        super().__init__()

        self.conv3x1_1 = nn.Conv2d(chann, chann, (3, 1), stride=1, padding=(1,0), bias=True)

        self.conv1x3_1 = nn.Conv2d(chann, chann, (1,3), stride=1, padding=(0,1), bias=True)

        self.bn1 = nn.BatchNorm2d(chann, eps=1e-03)

        self.conv3x1_2 = nn.Conv2d(chann, chann, (3, 1), stride=1, padding=(1*dilated,0), bias=True, dilation = (dilated,1))

        self.conv1x3_2 = nn.Conv2d(chann, chann, (1,3), stride=1, padding=(0,1*dilated), bias=True, dilation = (1, dilated))

        self.bn2 = nn.BatchNorm2d(chann, eps=1e-03)

        self.dropout = nn.Dropout2d(dropprob)


    def forward(self, input):

        output = self.conv3x1_1(input)
        output = F.relu(output)
        output = self.conv1x3_1(output)
        output = self.bn1(output)
        output = F.relu(output)

        output = self.conv3x1_2(output)
        output = F.relu(output)
        output = self.conv1x3_2(output)
        output = self.bn2(output)

        if (self.dropout.p != 0):
            output = self.dropout(output)

        return F.relu(output+input)    #+input = identity (residual connection)


class Encoder(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.initial_block = DownsamplerBlock(3,16)

        self.layers = nn.ModuleList()

        self.layers.append(DownsamplerBlock(16,64))

        for x in range(0, 5):    #5 times
           self.layers.append(non_bottleneck_1d(64, 0.03, 1))

        self.layers.append(DownsamplerBlock(64,128))

        for x in range(0, 2):    #2 times
            self.layers.append(non_bottleneck_1d(128, 0.3, 2))
            self.layers.append(non_bottleneck_1d(128, 0.3, 4))
            self.layers.append(non_bottleneck_1d(128, 0.3, 8))
            self.layers.append(non_bottleneck_1d(128, 0.3, 16))

        #Only in encoder mode:
        self.output_conv = nn.Conv2d(128, num_classes, 1, stride=1, padding=0, bias=True)

    def forward(self, input, predict=False):
        output = self.initial_block(input)

        for layer in self.layers:
            output = layer(output)

        if predict:
            output = self.output_conv(output)

        return output


class UpsamplerBlock (nn.Module):
    def __init__(self, ninput, noutput):
        super().__init__()
        self.conv = nn.ConvTranspose2d(ninput, noutput, 3, stride=2, padding=1, output_padding=1, bias=True)
        self.bn = nn.BatchNorm2d(noutput, eps=1e-3)

    def forward(self, input):
        output = self.conv(input)
        output = self.bn(output)
        return F.relu(output)

class Decoder (nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.layers = nn.ModuleList()

        self.layers.append(UpsamplerBlock(128,64))
        self.layers.append(non_bottleneck_1d(64, 0, 1))
        self.layers.append(non_bottleneck_1d(64, 0, 1))

        self.layers.append(UpsamplerBlock(64,16))
        self.layers.append(non_bottleneck_1d(16, 0, 1))
        self.layers.append(non_bottleneck_1d(16, 0, 1))

        self.output_conv = nn.ConvTranspose2d( 16, num_classes, 2, stride=2, padding=0, output_padding=0, bias=True)

    def forward(self, input):
        output = input

        for layer in self.layers:
            output = layer(output)

        output = self.output_conv(output)

        return output

#ERFNet
class ERFNet(nn.Module):
    def __init__(self, num_classes, encoder=None):  #use encoder to pass pretrained encoder
        super().__init__()

        if (encoder == None):
            self.encoder = Encoder(num_classes)
        else:
            self.encoder = encoder
        self.decoder = Decoder(num_classes)

    def forward(self, input, only_encode=False):
        if only_encode:
            return self.encoder.forward(input, predict=True)
        else:
            output = self.encoder(input)    #predict=False by default
            return self.decoder.forward(output)

In [ ]:
erfnet = ERFNet(num_classes = N_CLASSES_SEGMENTATION)

In [ ]:
class HybridLR(_LRScheduler):
    def __init__(self, optimizer, pow, max_iter, last_epoch=-1, warmup_epoch=0, min_lrs=1e-20):
        self.warmup_epoch = max(warmup_epoch, 0)
        self.pow = pow
        self.max_iter = max_iter

        self.min_lrs = min_lrs
        super(HybridLR, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        if self.last_epoch < self.warmup_epoch:
            return [base_lr / self.warmup_epoch * (self.last_epoch+1) for base_lr in self.base_lrs]
        if self.last_epoch < self.max_iter:
            coeff = (1 - (self.last_epoch-self.warmup_epoch) / (self.max_iter-self.warmup_epoch)) ** self.pow
        else:
            coeff = 0
        return [(base_lr - min_lr) * coeff + min_lr
                for base_lr, min_lr in zip(self.base_lrs, self.min_lrs)]

In [ ]:
def calculate_iou(pred, target, num_classes):
    pred_labels = torch.argmax(pred, dim=1)  # [batch_size, height, width]
    ious = []

    for cls in range(num_classes):
        # True Positive: Intersection
        intersection = ((pred_labels == cls) & (target == cls)).sum().item()

        # Union: (True Positive + False Positive + False Negative)
        union = ((pred_labels == cls) | (target == cls)).sum().item()

        if union == 0:
            iou = float('nan')  # Avoid division by zero
        else:
            iou = intersection / union
        ious.append(iou)

    mean_iou = torch.tensor(ious).nanmean()  # Compute the mean IoU, ignoring NaNs

    return ious, mean_iou

# Example usage
# Assuming pred is the model's output and target is the ground truth (both tensors are on the same device)
pred = torch.randn(8, 5, 360, 640)  # Example output tensor
target = torch.randint(0, 5, (8, 360, 640))  # Example ground truth with class indices


In [ ]:
# OLD LOSS
# class CombinedLoss(nn.Module):
#     def __init__(self, alpha=1.0, beta=1.0, gamma=1.0, epsilon=1e-6):
#         super(CombinedLoss, self).__init__()
#         self.alpha = alpha  # Weight for Cross-Entropy Loss
#         self.beta = beta    # Weight for BCE Loss
#         self.gamma = gamma  # Weight for Dice Loss
#         self.epsilon = epsilon

    # def forward(self, pred, target):
    #     # Get dimensions
    #     batchsize, classes, height, width = pred.size()

    #     # Cross-Entropy Loss (multi-class)
    #     ce_loss = F.cross_entropy(pred, target.long())

    #     # Convert target to one-hot encoding for BCE and Dice Loss
    #     target_one_hot = F.one_hot(target.long(), num_classes=NUM_CLASSES_SEGMENTATION).permute(0, 3, 1, 2).float()

    #     # Binary Cross-Entropy Loss
    #     bce_loss = F.binary_cross_entropy_with_logits(pred, target_one_hot)

    #     # Dice Loss Calculation
    #     pred_probs = F.softmax(pred, dim=1)  # Convert logits to probabilities
    #     smooth = self.epsilon
    #     intersection = (pred_probs * target_one_hot).sum(dim=(2, 3))
    #     union = pred_probs.sum(dim=(2, 3)) + target_one_hot.sum(dim=(2, 3))
    #     dice_loss = 1 - (2. * intersection + smooth) / (union + smooth)
    #     dice_loss = dice_loss.mean()  # Average over the batch

    #     # Combine the losses
    #     total_loss = self.alpha * ce_loss + self.beta * bce_loss + self.gamma * dice_loss
    #     return total_loss


In [ ]:
class CombinedLoss(nn.Module):
    def __init__(self):
        super(CombinedLoss, self).__init__()
        self.epsilon = 1e-9  # Small constant for numerical stability

    def forward(self, pred, target):
        batchsize, channel, height, width = pred.size()

        target_one_hot = None
        target= target.clamp(min=0, max=N_CLASSES_SEGMENTATION-1)
        # Ensure target has the correct shape
        if len(target.size()) == 3:
            target_one_hot = F.one_hot(target.long(), num_classes=N_CLASSES_SEGMENTATION).permute((0,3,1,2)).float()

            target = target.unsqueeze(dim=1)


        loss = 0.0 # Initial total loss

        # Convert logits to probabilities
        p = F.softmax(pred, dim=1)  # Probability map for the current sample

        # Loop over each sample in the batch
        for i in range(batchsize):

            ### Weighted Binary Cross Entripy Loss (WBCE Loss)
            # Background and foreground masks
            bg_mask = torch.eq(target[i], 0).float()  # Binary mask for background
            fg_mask = 1 - bg_mask  # Binary mask for foreground

            # Compute weights
            total_pixels = target[i].numel()  # Total number of pixels
            n_bg_pixels = bg_mask.sum()  # Number of background pixels
            n_fg_pixels = total_pixels - n_bg_pixels  # Number of foreground pixels

            bg_weight = n_bg_pixels / total_pixels  # Background weight
            fg_weight = 1 - bg_weight  # Foreground weight

            # Weighted BCE loss components

            bg_term = torch.sum(bg_weight * bg_mask * torch.log(p[i][0] + self.epsilon))
            fg_term = torch.sum(fg_weight * fg_mask * torch.log(torch.sum(p[i][1:], dim=0) + self.epsilon))

            # Normalize and combine terms
            wbce_loss = - (1 / total_pixels) * (bg_term + fg_term)

            ### DICE loss
            # Accumulate loss
            loss += wbce_loss
            # print(p[i].shape, target_one_hot[i].shape)

            intersection = (p[i] * target_one_hot[i]).sum(dim=(1,2))
            union = p[i].sum(dim=(1,2)) + target_one_hot[i].sum(dim=(1,2))

            dice_loss = 1 - torch.mean((2.0 * intersection + self.epsilon) / (union + self.epsilon))

            loss += dice_loss

        # Average loss over the batch
        return loss / batchsize


In [ ]:
def train_model(model, train_loader, val_loader, optimizer, loss_fn, num_epochs, device, num_classes):
    model.to(device)
    best_val_loss = float("inf")

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        model.train()

        running_loss = 0.0
        train_iou_scores = []

        for batch in tqdm(train_loader):
            # Load data
            images = batch["image"].to(device)
            lanes_image = batch["lanes_image"].to(device)

            # Zero gradients
            optimizer.zero_grad()

            # # Forward pass
            outputs = model(images)  # [batch_size, num_classes, H, W]

            # Compute loss
            loss = loss_fn(outputs, lanes_image)
            running_loss += loss.item()

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            # Calculate IoU
            # predicted_classes = torch.argmax(outputs, dim=1)  # Convert logits to class indices
            # print(predicted_classes.shape)  # Expected shape: [batch_size, height, width]

            _, batch_mean_iou = calculate_iou(outputs, lanes_image, num_classes)
            train_iou_scores.append(batch_mean_iou)

        avg_train_loss = running_loss / len(train_loader)
        avg_train_iou = sum(train_iou_scores) / len(train_iou_scores)
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_train_loss:.4f}, Train IoU: {avg_train_iou:.4f}")

        # Validation Step
        if val_loader is not None:
            val_loss, val_mean_iou = validate_model(model, val_loader, loss_fn, device, num_classes)
            print(f"Validation Loss: {val_loss:.4f}, Validation IoU: {val_mean_iou:.4f}")

            # Save the model if validation improves
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save(model.state_dict(), "erfnet_tusimple.pth")
                print("Saved Best Model")

def validate_model(model, val_loader, loss_fn, device, num_classes):
    model.eval()
    val_loss = 0.0
    val_iou_scores = []

    with torch.no_grad():
        for batch in val_loader:
            images = batch["image"].to(device)
            lanes_image = batch["lanes_image"].to(device)

            # Forward pass
            outputs = model(images)

            # Compute loss
            loss = loss_fn(outputs, lanes_image)
            val_loss += loss.item()

            # Calculate IoU
             # Expected shape: [batch_size, height, width]
            _, batch_mean_iou = calculate_iou(outputs, lanes_image, num_classes)
            val_iou_scores.append(batch_mean_iou)

    avg_val_iou = sum(val_iou_scores) / len(val_iou_scores)
    return val_loss / len(val_loader), avg_val_iou


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = optim.Adam(erfnet.parameters(), lr= LEARNING_RATE)
lr_scheduler = HybridLR(optimizer, 0.9, warmup_epoch= 20, max_iter= 30000)
loss_fn = CombinedLoss()

In [ ]:
train_model(erfnet, train_loader, val_loader, optimizer, loss_fn, N_EPOCHS, device, num_classes = N_CLASSES_SEGMENTATION)

Epoch 1/150


100%|██████████| 182/182 [00:52<00:00,  3.46it/s]

Epoch 1/150, Loss: 2.3201, Train IoU: 0.0750


Validation Loss: 2.2167, Validation IoU: 0.0945
Saved Best Model
Epoch 2/150


100%|██████████| 182/182 [00:50<00:00,  3.59it/s]

...

Epoch 149/150, Loss: 0.3375, Train IoU: 0.5571


Validation Loss: 0.4302, Validation IoU: 0.4810
Epoch 150/150


100%|██████████| 182/182 [00:51<00:00,  3.55it/s]

Epoch 150/150, Loss: 0.3372, Train IoU: 0.5574


Validation Loss: 0.4301, Validation IoU: 0.4810
